In [5]:
import psutil
import GPUtil
import threading
import time
import pandas as pd

def collect_psutil(metrics_list, additional_vals=None):
    if additional_vals is None:
        additional_vals = {}
    cpu_percent = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().percent
    disk_read_bytes = psutil.disk_io_counters().read_bytes
    disk_write_bytes = psutil.disk_io_counters().write_bytes
    network_bytes_sent = psutil.net_io_counters().bytes_sent
    network_bytes_received = psutil.net_io_counters().bytes_recv

    # Append metrics to the list
    row_dict = {
        'CPU Percent': cpu_percent,
        'Memory Usage': memory_usage,
        'Disk Read Bytes': disk_read_bytes,
        'Disk Write Bytes': disk_write_bytes,
        'Network Bytes Sent': network_bytes_sent,
        'Network Bytes Received': network_bytes_received
    }
    row_dict.update(additional_vals)
    metrics_list.append(row_dict)

class PsutilCollector:
    def __init__(self, interval=1):
        self.interval = interval
        self.data = []  # List to store collected metrics
        self.running = False
        self.thread = None

    def collect_metrics(self):
        while self.running:
            cpu_percent = psutil.cpu_percent(interval=self.interval)
            GPUs = GPUtil.getGPUs()
            gpu_utilization = GPUs[0].load
            memory_percent = psutil.virtual_memory().percent
            disk_read_bytes = psutil.disk_io_counters().read_bytes
            disk_write_bytes = psutil.disk_io_counters().write_bytes
            network_bytes_sent = psutil.net_io_counters().bytes_sent
            network_bytes_received = psutil.net_io_counters().bytes_recv

            # Append metrics to the list
            self.data.append({
                'CPU Percent': cpu_percent,
                'GPU Percent': gpu_utilization,
                'Memory Percent': memory_percent,
                'Disk Read Bytes': disk_read_bytes,
                'Disk Write Bytes': disk_write_bytes,
                'Network Bytes Sent': network_bytes_sent,
                'Network Bytes Received': network_bytes_received,
                "timestamp": time.time(),
            })
            per_cpu = psutil.cpu_percent(interval=self.interval, percpu=True)
            self.data[-1].update({f"cpu_{cpu}": percent for cpu, percent in enumerate(per_cpu)})

    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self.collect_metrics)
        self.thread.start()

    def stop(self):
        self.running = False
        self.thread.join()

    def create_dataframe(self):
        return pd.DataFrame(self.data)

# Example usage
if __name__ == "__main__":
    collector = PsutilCollector(interval=1)  # Set your desired interval
    collector.start()

    # Simulate experiment (replace with your actual experiment logic)
    time.sleep(2)  # Run the experiment for 10 seconds

    collector.stop()
    df = collector.create_dataframe()

    print("Collected metrics:")

    print(df)
    print(df.columns)


Collected metrics:
   CPU Percent  GPU Percent  Memory Percent  Disk Read Bytes  \
0          0.4         0.07            31.4      28234030080   

   Disk Write Bytes  Network Bytes Sent  Network Bytes Received     timestamp  \
0        9016268800            31508951               330470079  1.712059e+09   

   cpu_0  cpu_1  ...  cpu_22  cpu_23  cpu_24  cpu_25  cpu_26  cpu_27  cpu_28  \
0    0.0    0.0  ...     0.0     0.0     1.6     0.0     0.0     0.0     0.0   

   cpu_29  cpu_30  cpu_31  
0     0.0     1.6     1.6  

[1 rows x 40 columns]
Index(['CPU Percent', 'GPU Percent', 'Memory Percent', 'Disk Read Bytes',
       'Disk Write Bytes', 'Network Bytes Sent', 'Network Bytes Received',
       'timestamp', 'cpu_0', 'cpu_1', 'cpu_2', 'cpu_3', 'cpu_4', 'cpu_5',
       'cpu_6', 'cpu_7', 'cpu_8', 'cpu_9', 'cpu_10', 'cpu_11', 'cpu_12',
       'cpu_13', 'cpu_14', 'cpu_15', 'cpu_16', 'cpu_17', 'cpu_18', 'cpu_19',
       'cpu_20', 'cpu_21', 'cpu_22', 'cpu_23', 'cpu_24', 'cpu_25', 'cpu_26',

In [6]:
def close_envs(envs: list, prints=False):
    envs_length = len(envs)
    for i in range(envs_length):
        env = envs.pop()  # Also clears the given envs list, so we do not need to separately clear it
        try:
            env.close()  # Might raise exception if already closed
            if prints:
                print("Closed env")
        except:
            if prints:
                print("Could not close")
    
    

In [7]:

# import gym
import os
from matplotlib import pyplot as plt
from ray.tune import tune
from ray_utils.unity_env import BetterUnity3DEnv
import concurrent.futures

# List of the number of audio sources used in the experiments
audio_sources_to_test = [1, 10, 30]

# Path to unity build
unity_build_path = "../builds/aaaa-windows/AA-Agent-Avoidance.exe"

# Folder for measurement results
output_folder = "outputs/exp_1"
os.makedirs(output_folder, exist_ok=True)

# MLAgents decision period
# Using decision period 10 is a reasonable assumption, since the audio buffer takes 10 steps to fill up.
# Decision period 1 can also be used, but then the agents will make decisions with overlapping information, which can be unnecessary and inefficient.
decision_period = 10

# Run the benchmarks
for audio_sources in audio_sources_to_test:
    run_name = f"{audio_sources}"
    
    unity_log_folder = os.path.join(os.getcwd(), output_folder, f"unity_logs_{audio_sources}")  # Absolute path for unity logs for debugging, use None to disable
    
    agent = "hanningAO"  # Should not matter too much which agent is being used, since most of the load comes from the number of audio sources.
    args = ["-audioSources", f"{audio_sources}", "-decisionPeriod", f"{decision_period}", "-agent", agent]
    
    collector = PsutilCollector(interval=1)  # Set your desired interval
    collector.start()
    result_rows = []
    envs = []
    try:
        delete_envs_after_every_run = False
        for num_envs in range(1,41, 1):
            time.sleep(3) # Easier to sync psutil metrics with some delay between runs
            while len(envs) < num_envs:
                try:
                    new_env = BetterUnity3DEnv(file_name=unity_build_path, no_graphics=True, args=args, log_folder=unity_log_folder)
                    new_env.step({})
                    envs.append(new_env)
                except Exception as e:
                    print(f"Could not create a new env - target:{num_envs} and len: {len(envs)}")
                    print(e)
                    close_envs(envs, prints=True)
                    exit(1)
                # envs = [BetterUnity3DEnv(file_name=unity_build_path, no_graphics=True) for i in range(num_envs)]
            [e.reset() for e in envs]
            [e.step({}) for e in envs]
            time.sleep(3) # Easier to sync psutil metrics with some delay between runs
            times = []
            x = []
            
            with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
                # collect_psutil(psutil_rows, {"num_envs": 0, "num_iters": 0})
                
                for i in (20,):
                    start_time = time.time()
                    iters = i
                    def measure_env(env):
                        """
                        This measures the time it takes to execute 'iters' amount of full steps 
                        (one full step depends on decision period) in one environment.
                        
                        When plotting results, it is necessary to account for the possibility that all environments were not
                        executed in parallel. In this sense, the timestamps collected in this method can not be fully trusted.
                        
                        For example, there might be two batches of 20 environments, which would give the time it takes to run
                        20 environments instead of 40 environments in parallel. Using a separate time stamp (start_time),
                        which is computed outside this method can fix this issue.
                        """
                        t1 = time.time()
                        for _ in range(iters):
                            # result = timeit.timeit(lambda: env.step({}), number=iters)
                            result = env.step({})
                        t2 = time.time()
                        return t2 - t1
                    future_to_index = {executor.submit(measure_env, envs[index]): index for index in range(len(envs))}
                    for future in concurrent.futures.as_completed(future_to_index):
                        result = future.result()
                        index = future_to_index[future]
                        print(f"completed {result}")
        
                        times.append(result)
                        x.append(iters)
                        result_rows.append({"num_envs": num_envs, "env_index": index, "num_iters": iters, "exec_time": result, "start_timestamp": start_time, "timestamp": time.time()})

            pd.DataFrame(result_rows).to_feather(f"{output_folder}/unity_{run_name}.feather")
            collector.create_dataframe().to_feather(f"{output_folder}/psutil_{run_name}.feather")
            print(f"Saved num_envs {num_envs} to {output_folder}/unity_{run_name}")
            if delete_envs_after_every_run:
                close_envs(envs, prints=False)
                envs = []
    except Exception as e:
        print(e)
        print("Fell to wide try-except")
        close_envs(envs, prints=True)
    
    close_envs(envs, prints=False)
    
    time.sleep(5) # Let psutil collector collect the tail for reference
    collector.stop()
    df = pd.DataFrame(result_rows)
    psutil_df = collector.create_dataframe()
    df.to_feather(f"{output_folder}/unity_{run_name}.feather")
    psutil_df.to_feather(f"{output_folder}/psutil_{run_name}.feather")
    # psutil_df = pd.DataFrame(psutil_rows)



Seed: 3660
Created UnityEnvironment for port 53565
x
completed 3.9907243251800537
Saved num_envs 1 to outputs/exp_1/unity_10_intermediate
Seed: 992
Created UnityEnvironment for port 50897
x
completed 3.98494291305542
completed 3.990973472595215
Saved num_envs 2 to outputs/exp_1/unity_10_intermediate
Seed: 9434
Created UnityEnvironment for port 59339
x
completed 3.9851460456848145
completed 3.987138271331787
completed 3.9991843700408936
Saved num_envs 3 to outputs/exp_1/unity_10_intermediate
Seed: 9604
Created UnityEnvironment for port 59509
x
completed 3.9857177734375
completed 3.9867284297943115
completed 3.9867324829101562
completed 3.9877634048461914
Saved num_envs 4 to outputs/exp_1/unity_10_intermediate
Seed: 1936
Seed: 4898
Created UnityEnvironment for port 54803
x
completed 3.9845337867736816
completed 3.986036777496338
completed 3.990556240081787
completed 3.98905348777771
completed 3.9948155879974365
Saved num_envs 5 to outputs/exp_1/unity_10_intermediate
Seed: 4116
Created Un